In [8]:
!pip install -q --upgrade pip
!pip install -q tensorflow-gpu==2.2.0
!pip install -q transformers

     |████████████████████████████████| 1.5MB 8.8MB/s 
     |████████████████████████████████| 516.2 MB 19 kB/s 
     |████████████████████████████████| 674 kB 7.7 MB/s 
     |████████████████████████████████| 883 kB 23.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.8 MB/s 
     |████████████████████████████████| 3.8 MB 73.7 MB/s 


In [9]:
!git clone https://github.com/rgrupesh/Deep_Learning.git

Cloning into 'Deep_Learning'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 68 (delta 27), reused 27 (delta 7), pack-reused 0
Unpacking objects: 100% (68/68), done.


In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [0]:
train=pd.read_csv("/content/Deep_Learning/data/SST-5/sst_train.txt", sep="\t",header=None, names=["label", "text"])
train=train[["text","label"]]
train.label=train.label-1


val=pd.read_csv("/content/Deep_Learning/data/SST-5/sst_dev.txt", sep="\t",header=None, names=["label", "text"])
val=val[["text","label"]]
val.label=val.label-1

test=pd.read_csv("/content/Deep_Learning/data/SST-5/sst_test.txt", sep="\t",header=None, names=["label", "text"])
test=test[["text","label"]]
test.label=test.label-1

In [99]:
train.tail()

,text,label
8539,A real snooze .,0
8540,No surprises .,1
8541,We 've seen the hippie-turned-yuppie plot befo...,3
8542,Her fans walked out muttering words like `` ho...,0
8543,In this case zero .,1


In [0]:
records_train = train.to_records(index=False)
result_train = list(records_train)

records_val = val.to_records(index=False)
result_val = list(records_val)

records_test = test.to_records(index=False)
result_test = list(records_test)


In [0]:
from transformers import TFXLNetForSequenceClassification, XLNetConfig, XLNetTokenizer
from transformers import *

In [0]:
MAX_LENGTH=512
BATCH_SIZE=6
EVAL_BATCH_SIZE=12

NUM_LABELS=5

config=XLNetConfig.from_pretrained("xlnet-base-cased", num_labels=NUM_LABELS,ff_activation="gelu",dropout=0.1)
tokenizer=XLNetTokenizer.from_pretrained("xlnet-base-cased")
model= TFXLNetForSequenceClassification.from_pretrained("xlnet-base-cased",config=config)

In [124]:
config

XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "untie_r": true,
  "vocab_size": 32000

In [0]:
def convert_example_to_feature(text):
  
  return tokenizer.encode_plus(text, 
                add_special_tokens = True, 
                max_length = MAX_LENGTH, 
                pad_to_max_length = True, 
                return_attention_mask = True,
              )

In [0]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds):
  
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

    
  for text, label in ds:

    bert_input = convert_example_to_feature(text)
  
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

In [0]:
ds_train_encoded = encode_examples(result_train).shuffle(10000).batch(BATCH_SIZE)
ds_val_encoded = encode_examples(result_val).shuffle(10000).batch(EVAL_BATCH_SIZE)
ds_test_encoded = encode_examples(result_test).shuffle(10000).batch(EVAL_BATCH_SIZE)

In [128]:
print(ds_train_encoded)

<BatchDataset shapes: ({input_ids: (None, 512), token_type_ids: (None, 512), attention_mask: (None, 512)}, (None, 1)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>


In [0]:
optimizer=tf.keras.optimizers.Adam(lr=5e-5,epsilon=1e-8)
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric=tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

model.compile(optimizer=optimizer,loss=loss,metrics=[metric])

In [0]:
train_steps= len(result_train) // BATCH_SIZE
val_steps= len(result_val) // EVAL_BATCH_SIZE

In [131]:
model.summary()

Model: "tfxl_net_for_sequence_classification_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (TFXLNetMainLaye multiple                  116718336 
_________________________________________________________________
sequence_summary (TFSequence multiple                  590592    
_________________________________________________________________
logits_proj (Dense)          multiple                  3845      
Total params: 117,312,773
Trainable params: 117,312,773
Non-trainable params: 0
_________________________________________________________________


In [0]:
history= model.fit(ds_train_encoded, epochs=1, validation_data=ds_val_encoded, verbose=2, steps_per_epoch=train_steps, validation_steps=val_steps)

In [0]:
history1=model.evaluate(ds_test_encoded)